In [10]:
!pip install --upgrade transformers SPARQLWrapper

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.1 MB/s eta 0:00:00
  Obtaining dependency information for rdflib>=6.1.1 from https://files.pythonhosted.org/packages/d4/b0/7b7d8b5b0d01f1a0b12cc2e5038a868ef3a15825731b8a0d776cf47566c0/rdflib-7.0.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 34.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [11]:
from transformers import pipeline

In [17]:
oracle = pipeline(model="deepset/roberta-base-squad2")
print(oracle(question="What is the capital of Germany?", context="Germany is a country in Europe. The capital is Berlin."))

{'score': 2.9879427529522218e-05, 'start': 0, 'end': 31, 'answer': 'Germany is a country in Europe.'}


In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON
def get_context(entity):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
    SELECT ?abstract
        WHERE { 
            <http://dbpedia.org/resource/%s> dbo:abstract ?abstract .
            FILTER (lang(?abstract) = 'en') .
        }
    """ % entity)
    print(sparql.query())
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    context = [result["abstract"]["value"] for result in results["results"]["bindings"]]
    return context

In [14]:
ner = pipeline("ner")
question="What is the capital of Germany?"
entities = ner(question)
entity = entities[0]["word"]

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
oracle = pipeline(model="deepset/roberta-base-squad2")
question="What is the capital of Germany?"
# Use the function to get information on entity
context=get_context(entity)[0]

{"requestedFormat" : 'xml',
"response (a file-like object, as return by the urllib2.urlopen library call)" : {
	"url" : "https://dbpedia.org/sparql?query=%0A++++SELECT+%3Fabstract%0A++++++++WHERE+%7B+%0A++++++++++++%3Chttp%3A//dbpedia.org/resource/Germany%3E+dbo%3Aabstract+%3Fabstract+.%0A++++++++++++FILTER+%28lang%28%3Fabstract%29+%3D+%27en%27%29+.%0A++++++++%7D%0A++++&format=xml&output=xml&results=xml",
	"code" : "200",
	"headers" : Date: Mon, 04 Dec 2023 09:02:18 GMT
Content-Type: application/sparql-results+xml; charset=UTF-8
Content-Length: 3646
Connection: close
Server: Virtuoso/08.03.3329 (Linux) x86_64-generic-linux-glibc25  VDB
Content-disposition: filename=sparql_2023-12-04_09-02-20Z.xml
Expires: Mon, 11 Dec 2023 09:02:18 GMT
Cache-Control: max-age=604800
Access-Control-Allow-Credentials: true
Access-Control-Allow-Methods: GET, POST, OPTIONS
Access-Control-Allow-Headers: Depth,DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Typ

In [16]:
print(oracle(question=question, context=context)["answer"])

Berlin
